In [ ]:
# TODO 
# 1. create tiles (trio.ipynb > get_tiles())
# 2. for each sample 
# 3. average cpg methylation on each hap in each tile (some values may be null; what does average do wih these?) (trio.ipynb > get_methylation_trio_by_interval() or recombination.py > compute_methylation())
# 4. join tiles over all samples, keeping meth_pat, meth_mat, delta_meth and hap_pat, hap_mat  
# 5. filter tiles for those in which delta_meth is close to 1 in at least one sample 
# 6. is delta_meth the same for all samples? is meth_pat and meth_mat the same over all samples? compare with "truth set" of imprinted loci here: https://quinlangroup.slack.com/archives/C0803TM7X0X/p1758742312373849
# 7. are there polymorphic loci, where delta-meth varies over samples? and if so, does meth_pat, meth_mat, delta_meth covary with founder haplotype of degree of relationship? 
